# Accumulators

## Syntax & Correctness

An accumulator is a compact representation of a set of elements and allows certain operations on the set.

:::{note} Definition: Accumulator
:label: def-accumulator

An accumulator that supports insertions of elements and membership proofs is also called an *additive positive accumulator*.
It is a tuple of polynomial-time algorithms $(\mathsf{Gen}, \mathsf{Init}, \mathsf{Value}, \mathsf{Insert}, \mathsf{ProveMembership}, \mathsf{VerifyMembership})$ where:

- $\mathsf{Gen}(1^\lambda) \rightarrow \mathit{pp}$ is a probabilistic algorithm that takes the security parameter as input and outputs public parameters $\mathit{pp}$ including the domain $\mathcal{X}_\mathsf{Acc}$ of elements that can be accumulated.
- $\mathsf{Init}(\mathit{pp}) \rightarrow \mathit{state}$ is a deterministic algorithm that takes public parameters $\mathit{pp}$ as input and outputs the initial state, $\mathit{state}$, of the accumulator.
- $\mathsf{Value}(\mathit{pp}, \mathit{state}) \rightarrow v$ is a deterministic algorithm that takes public parameters $\mathit{pp}$ and an accumulator state, $\mathit{state}$, as input and outputs the accumulator value $v$.
- $\mathsf{Insert}(\mathit{pp}, \mathit{state}, x) \rightarrow \mathit{state}'$ is a deterministic algorithm that takes public parameters $\mathit{pp}$, an accumulator state, $\mathit{state}$, and an element $x \in \mathcal{X}_\mathsf{Acc}$ as input and outputs a new state, $\mathit{state}'$, after inserting $x$.
- $\mathsf{ProveMembership}(\mathit{pp}, \mathit{state}, x) \rightarrow \pi$ is a probabilistic algorithm that takes public parameters $\mathit{pp}$, an accumulator state, $\mathit{state}$, and an element $x$ as input and outputs a membership proof $\pi$.
- $\mathsf{VerifyMembership}(\mathit{pp}, v, x, \pi) \rightarrow \{0, 1\}$ is a deterministic algorithm that takes public parameters $\mathit{pp}$, an accumulator value $v$, an element $x \in \mathcal{X}_\mathsf{Acc}$, and a proof $\pi$ as input and outputs $1$ (accept) if the proof validates that $x$ is a member of the set represented by $v$, and outputs $0$ (reject) otherwise.
:::

Typically, an accumulator involves two parties: the manager, who uses $\mathsf{Init}$, $\mathsf{Value}$, $\mathsf{Insert}$, $\mathsf{ProveMembership}$ and the verifier, who uses $\mathsf{VerifyMembership}$.

:::{tip} Example: Trivial Accumulator
:label: sec-trivial-acc

An example is the trivial accumulator, where the accumulator value is simply the set itself and therefore grows linearly with the number of inserted elements.
(In contrast, non-trivial accumulators typically have constant-size accumulator values.)

- $\mathsf{Gen}(1^\lambda)\rightarrow \mathit{pp}$ where $\mathit{pp} = (\mathcal{X}_\mathsf{Acc} = \{0,1\}^*)$
- $\mathsf{Init}(\mathit{pp})\rightarrow \{\}$
- $\mathsf{Value}(\mathit{pp}, \mathit{state})\rightarrow \mathit{state}$
- $\mathsf{Insert}(\mathit{pp}, \mathit{state}, x) \rightarrow \mathit{state} \cup \{x\}$
- $\mathsf{ProveMembership}(\mathit{pp}, \mathit{state}, x) \rightarrow \bot$
- $\mathsf{VerifyMembership}(\mathit{pp}, v, x, \pi) \rightarrow b$ outputs $b = 1$ if $x \in v$. Otherwise, it outputs $b = 0$.
:::

:::{note} Definition: Correctness
:label: def-accumulator-correctness

An additive positive accumulator is considered *correct* if, for any element that was inserted into the accumulator, the membership proof generated honestly for it is successfully verified.
More formally, for all security parameters $\lambda$, all public parameters $\mathit{pp} \leftarrow \mathsf{Gen}(1^\lambda)$, all positive integers $t$ polynomial in $\lambda$, all tuples $\vec{y} \in \mathcal{X}_\mathsf{Acc}^{t}$: for all $x \in \vec{y}$,

$$
\Pr\left[
\begin{array}{l}
    \mathit{state}_0 \leftarrow \mathsf{Init}(\mathit{pp}); \\
    \mathit{state}_i \leftarrow \mathsf{Insert}(\mathit{pp}, \mathit{state}_{i - 1}, y_i) \text{ for } i \in [1, t]; \\
    \pi \leftarrow \mathsf{ProveMembership}(\mathit{pp}, \mathit{state}_t, x); \\
    v := \mathsf{Value}(\mathit{pp}, \mathit{state}_t); \\
    \mathsf{VerifyMembership}(\mathit{pp}, v, x, \pi) = 1
\end{array}
\right] = 1 - \mathrm{negl}.
$$
:::

## Security

In $\mathsf{Game}~\mathsf{AccColl}$, the adversary $\mathcal{A}$ is given access to *oracles*, which he can call, but not look inside.
In other words, the adversary can insert elements into the accumulator and obtain membership proofs, but does not have access to the state directly.

:::{note} Game: Collision-Freeness
:label: fig-break-acc-coll

$$
\begin{array}{l}
\underline{\text{Game } \mathsf{AccColl}^{\mathsf{Acc}}}:\\
\quad \mathit{pp} \leftarrow_R \mathsf{Gen}(1^\lambda) \\
\quad \mathcal{Q} \leftarrow \emptyset \\
\quad \mathit{state} \leftarrow \mathsf{Init}(\mathit{pp}) \\
\quad v \leftarrow \mathsf{Value}(\mathit{pp}, \mathit{state}) \\
\quad (x^*, \pi^*) \leftarrow \mathcal{A}^{\mathsf{Insert}, \mathsf{ProveMembership}}(\mathit{pp}, v) \\
\quad \text{return } x^* \notin \mathcal{Q} \wedge \mathsf{VerifyMembership}(\mathit{pp}, v, x^*, \pi^*) = 1
\end{array}
$$

**Oracle $\mathsf{Insert}(x)$:**
$$
\begin{array}{l}
\mathcal{Q} \leftarrow \mathcal{Q} \cup \{x\} \\
\mathit{state} \leftarrow \mathsf{Insert}(\mathit{pp}, \mathit{state}, x) \\
\text{// update global } v \\
v \leftarrow \mathsf{Value}(\mathit{pp}, \mathit{state}) \\
\text{return } v
\end{array}
$$

**Oracle $\mathsf{ProveMembership}(x)$:**
$$
\begin{array}{l}
\text{if } x \in \mathcal{Q} \text{ then}\\
\quad \text{return } \mathsf{ProveMembership}(\mathit{pp}, \mathit{state}, x) \\
\text{else}\\
\quad \text{return } \bot
\end{array}
$$
:::

:::{note} Definition: Collision-Freeness
:label: def-collision-freeness

An additive positive accumulator is *collision-free* if it is hard to fabricate a membership proof for a value that is not in the set.
More formally, an accumulator $\mathsf{Acc}$ is collision-free if for any PPT algorithm $\mathcal{A}$

$$
\mathcal{A}antage{Collision}{\mathcal{A}, \mathsf{Acc}} := \Pr[\mathsf{AccColl}_{\mathsf{Acc}} = 1} = \mathrm{negl}.
$$
:::

## Exercises

:::{exercise}
:label: ex-trivial-acc-correct

Is the trivial accumulator correct?
Justify or prove it.
:::

:::{dropdown} **Solution**

*Correctness (informal):* An accumulator is correct if for any element that has been inserted, we can generate a membership proof (using the current state) that will verify successfully against the current accumulator value.

*Is the trivial accumulator correct?* Yes, the trivial accumulator is correct.
In the trivial accumulator:

- The accumulator value $v$ is just the state itself, which is the set of all inserted elements
- $\mathsf{ProveMembership}$ returns $\bot$ (no proof needed)
- $\mathsf{VerifyMembership}$ simply checks if $x \in v$

Since the accumulator value is the set of all inserted elements, any element that was inserted will be in the set, so verification always succeeds for inserted elements.
This satisfies correctness.
:::

:::{exercise}
:label: ex-trivial-acc-collision

Is the trivial accumulator collision-free?
Justify or prove it.
:::

:::{dropdown} **Solution**

*Collision-freeness (informal):* An accumulator is collision-free if it's computationally infeasible for an adversary to produce a valid membership proof for an element that was never inserted into the accumulator.

*Is the trivial accumulator collision-free?* Yes, the trivial accumulator is collision-free.

In the trivial accumulator, the accumulator value $v$ is exactly the set of all inserted elements.
For $\mathsf{VerifyMembership}$ to accept, the element $x$ must be in this set.
Since the adversary cannot modify the accumulator value (it's computed honestly by the game), and the value contains exactly the elements that were inserted, it's impossible to create a proof for an element that wasn't inserted.

Therefore, $\mathcal{A}antage{Collision}{\mathcal{A}, \mathsf{TrivialAcc}} = 0$ for all adversaries, making it perfectly collision-free.
:::

:::{exercise}
:label: ex-crhf-accumulator

Using a collision-resistant hash function, give an example of an additive positive accumulator with constant-size accumulator values.
Argue that it is correct and collision-free.
:::

:::{dropdown} **Solution**

An example is a *Merkle tree*.
The accumulator value is the root hash of a Merkle tree containing all inserted elements.
The membership proof for an element consists of the authentication path from the leaf to the root.

*Correctness:* If an element was inserted, it appears as a leaf in the tree.
The honestly generated authentication path will correctly hash up to the root, so verification succeeds.

*Collision-freeness:* To forge a membership proof for an element that wasn't inserted, an adversary would need to provide an authentication path that hashes to the correct root.
This would mean finding a different set of sibling hashes that produce the same root hash, which would constitute a collision in the hash function.
Since the hash function is collision-resistant, this is infeasible.
:::

:::{exercise}
:label: ex-log-size-proofs

Using a collision-resistant hash function, give an example of an additive positive accumulator with constant-size accumulator values and logarithmic-size membership proofs.
Argue that it is correct and collision-free. (Optional)
:::

:::{dropdown} **Solution**

A Merkle tree achieves logarithmic-size proofs.
Elements are stored at the leaves of a balanced binary tree, and the accumulator value is the root hash.
The membership proof consists of the sibling hashes along the path from leaf to root, which has length $O(\log n)$ for $n$ elements.

*Correctness and Collision-freeness:* Same as the previous exercise.
:::

:::{exercise}
:label: ex-universal-accumulator

Define the syntax, correctness, and collision-freeness of an additive universal accumulator (i.e., an additive positive accumulator that supports non-membership proofs).
:::

:::{dropdown} **Solution**

*Syntax:* An additive universal accumulator is a tuple of polynomial-time algorithms

$$
\begin{align*}
&(\mathsf{Gen}, \mathsf{Init}, \mathsf{Value}, \mathsf{Insert}, \mathsf{ProveMembership}, \mathsf{VerifyMembership},\\
&\quad \mathsf{ProveNonMembership}, \mathsf{VerifyNonMembership}),
\end{align*}
$$

where the first six algorithms are as in the additive positive accumulator, and:

- $\mathsf{ProveNonMembership}(\mathit{pp}, \mathit{state}, x) \rightarrow \pi$ is a probabilistic algorithm that takes public parameters $\mathit{pp}$, an accumulator state, $\mathit{state}$, and an element $x$ as input and outputs a non-membership proof $\pi$.
- $\mathsf{VerifyNonMembership}(\mathit{pp}, v, x, \pi) \rightarrow \{0, 1\}$ is a deterministic algorithm that takes public parameters $\mathit{pp}$, an accumulator value $v$, an element $x \in \mathcal{X}_\mathsf{Acc}$, and a proof $\pi$ as input and outputs $1$ (accept) if the proof validates that $x$ is not a member of the set represented by $v$, and outputs $0$ (reject) otherwise.

*Correctness:* An additive universal accumulator is *correct* if both membership and non-membership proofs verify correctly.
Formally, for all security parameters $\lambda$, all public parameters $\mathit{pp} \leftarrow \mathsf{Gen}(1^\lambda)$, all values $x \in \mathcal{X}_\mathsf{Acc}$, all positive integers $t$ polynomial in $\lambda$, all tuples $\vec{y} \in \mathcal{X}_\mathsf{Acc}^{t}$:

1. Membership correctness: For all $x \in \vec{y}$,

$$
\Pr\left[
\begin{array}{l}
  \mathit{state}_0 \leftarrow \mathsf{Init}(\mathit{pp}); \\
  \mathit{state}_i \leftarrow \mathsf{Insert}(\mathit{pp}, \mathit{state}_{i-1}, y_i) \text{ for } i \in [1, t]; \\
  \pi \leftarrow \mathsf{ProveMembership}(\mathit{pp}, \mathit{state}_t, x); \\
  v := \mathsf{Value}(\mathit{pp}, \mathit{state}_t); \\
  \mathsf{VerifyMembership}(\mathit{pp}, v, x, \pi) = 1
\end{array}
\right] = 1 - \mathrm{negl}.
$$

2. Non-membership correctness: For all $x \notin \vec{y}$,

$$
\Pr\left[
\begin{array}{l}
  \mathit{state}_0 \leftarrow \mathsf{Init}(\mathit{pp}); \\
  \mathit{state}_i \leftarrow \mathsf{Insert}(\mathit{pp}, \mathit{state}_{i-1}, y_i) \text{ for } i \in [1, t]; \\
  \pi \leftarrow \mathsf{ProveNonMembership}(\mathit{pp}, \mathit{state}_t, x); \\
  v := \mathsf{Value}(\mathit{pp}, \mathit{state}_t); \\
  \mathsf{VerifyNonMembership}(\mathit{pp}, v, x, \pi) = 1
\end{array}
\right] = 1 - \mathrm{negl}.
$$

*Collision-Freeness:* An additive universal accumulator is *collision-free* if it's hard to fabricate either a membership proof for a non-member or a non-membership proof for a member.

$$
\begin{array}{l}
\underline{\text{Game } \mathsf{UnivAccColl}^{\mathsf{Acc}}}:\\
\quad \mathit{pp} \leftarrow_R \mathsf{Gen}(1^\lambda) \\
\quad \mathcal{Q} \leftarrow \emptyset \\
\quad \mathit{state} \leftarrow \mathsf{Init}(\mathit{pp}) \\
\quad v \leftarrow \mathsf{Value}(\mathit{pp}, \mathit{state}) \\
\quad (x^*, \pi^*, b^*) \leftarrow \mathcal{A}^{\mathsf{Insert}, \mathsf{ProveMembership}, \mathsf{ProveNonMembership}}(\mathit{pp}, v) \\
\quad \text{if } b^* = 1 \text{ then} \\
\quad\quad \text{return } x^* \notin \mathcal{Q} \wedge \mathsf{VerifyMembership}(\mathit{pp}, v, x^*, \pi^*) = 1 \\
\quad \text{else} \\
\quad\quad \text{return } x^* \in \mathcal{Q} \wedge \mathsf{VerifyNonMembership}(\mathit{pp}, v, x^*, \pi^*) = 1
\end{array}
$$

The oracles $\mathsf{Insert}$ and $\mathsf{ProveMembership}$ are as before, and $\mathsf{ProveNonMembership}(x)$ returns $\mathsf{ProveNonMembership}(\mathit{pp}, \mathit{state}, x)$ if $x \notin \mathcal{Q}$ and $\bot$ otherwise.

An accumulator $\mathsf{Acc}$ is collision-free if for any PPT algorithm $\mathcal{A}$:

$$
\mathcal{A}antage{UnivColl}{\mathcal{A}, \mathsf{Acc}} := \Pr[\mathsf{UnivAccColl}_{\mathsf{Acc}} = 1} = \mathrm{negl}.
$$
:::

:::{exercise}
:label: ex-universal-acc-construction

Using a collision-resistant hash function, give an example of an additive universal accumulator with constant-size accumulator values and logarithmic-size membership and non-membership proofs.
Argue that it is correct and collision-free. (Optional)
:::

:::{dropdown} **Solution**

We can construct a universal accumulator using a *sparse Merkle tree*.
In a sparse Merkle tree, we have a fixed domain $\mathcal{X}_\mathsf{Acc} = \{0,1\}^\lambda$ and build a complete binary tree of depth $\lambda$.
Each element $x \in \mathcal{X}_\mathsf{Acc}$ maps to a unique leaf position determined by interpreting $x$ as a binary path.

*Construction:*

- $\mathsf{Gen}(1^\lambda)$: Output $\mathit{pp} = (\mathcal{X}_\mathsf{Acc} = \{0,1\}^\lambda, \mathsf{H})$ where $\mathsf{H}$ is a collision-resistant hash function.
- $\mathsf{Init}(\mathit{pp})$: Initialize an empty sparse Merkle tree where all leaves have a default value $\bot$ (representing non-membership).
- $\mathsf{Value}(\mathit{pp}, \mathit{state})$: Return the root hash of the sparse Merkle tree.
- $\mathsf{Insert}(\mathit{pp}, \mathit{state}, x)$: Set the leaf at position $x$ to a non-$\bot$ value (e.g., $\mathsf{H}(x)$) and update the tree.
- $\mathsf{ProveMembership}(\mathit{pp}, \mathit{state}, x)$: Return the authentication path from leaf $x$ to the root.
- $\mathsf{VerifyMembership}(\mathit{pp}, v, x, \pi)$: Verify that the path leads to root $v$ and the leaf value is non-$\bot$.
- $\mathsf{ProveNonMembership}(\mathit{pp}, \mathit{state}, x)$: Return the authentication path showing the leaf at position $x$ has value $\bot$.
- $\mathsf{VerifyNonMembership}(\mathit{pp}, v, x, \pi)$: Verify that the path leads to root $v$ and the leaf value is $\bot$.

*Efficiency:* The accumulator value (root hash) has constant size.
Both membership and non-membership proofs consist of $\lambda$ sibling hashes along the path, giving logarithmic proof size.

*Correctness:*

- Membership: If $x$ was inserted, its leaf has a non-$\bot$ value, and the authentication path correctly verifies to the root.
- Non-membership: If $x$ was never inserted, its leaf remains $\bot$, and the authentication path proves this.

*Collision-freeness:* To forge a membership proof for a non-member (or vice versa), an adversary would need to provide an authentication path that:

- Hashes to the correct root value
- Shows a different leaf value than what's actually stored

This would require finding a collision in the hash function, which is infeasible by assumption.

*Alternative:* A sorted Merkle tree could also be used, where elements are stored in sorted order at the leaves.
Non-membership proofs would show two adjacent leaves with values that bracket the queried element.
:::

:::{exercise}
:label: ex-strength-notion

Read Section 2 of Baldimtsi, Canetti, and Yakoubov.
Describe the notion of *Strength* for accumulators. (Optional)
:::

:::{exercise}
:label: ex-shielded-csv

Read Sections 3.5, 3.6, and Appendix A.2 of Shielded CSV. (Optional)

1. How do the accumulators required for Shielded CSV differ from the accumulators discussed in this section?
2. How would you approach proving the $\mathsf{ToSA\textnormal{-}SEC}$ security property (an open problem at the time of writing)?
:::

:::{exercise}
:label: ex-security-notions

Read Section 5 of Cremer, Loss, and Wagner, which provides security notions for various cryptographic primitives.
Summarize the key insights. (Optional)
:::

:::{exercise}
:label: ex-utreexo

Characterize the Utreexo Accumulator. (Optional)
:::